# Case 2 Part 2: Anomaly Detection

This notebook showcases the 2nd part of the analytics use case to be tackled in a real-time alerting system:

`Hourly consumption for a household is higher than 1 standard deviation of mean consumption across all households within that particular hour on that day.`

The second part here is to read the data stream from both `readings_prepared` and `alert_2_stats` and use them to detect data anomalies -- the ones that go above 1 standard deviation from the mean. The detected anomalies are then stored in a persistent data store, which in this case is Google BigQuery.

BigQuery is chosen for the fit of further analytical queries down the line. We might be interested to do some BI or advanced analysis down the line. It is also serverless -- we just need to define the Datasets and Tables.


## Setup

Import all the required libraries and set the stream configuration variables.

In [1]:
import spark.implicits._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql._

Intitializing Scala interpreter ...

Spark Web UI available at http://spark-alert-2-detect-m:8088/proxy/application_1583313187361_0001
SparkContext available as 'sc' (version = 2.4.5, master = yarn, app id = application_1583313187361_0001)
SparkSession available as 'spark'


import spark.implicits._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql._


In [2]:
val kafkaBootstrapServer = "kafka-m:9092"
val kafkaReadingsTopic = "readings_prepared"
val kafkaStatsTopic = "alert_2_stats"
val kafkaDedupWatermarkTime = "1 minute"
val joinWatermarkTime = "1 minute"
val bigQueryTargetTable = "smartplugs.alert_2_anomaly"
val bigQueryTempBucket = "pandora-sde-case/alert_2"
val outputTriggerTime = "1 minute"

kafkaBootstrapServer: String = kafka-m:9092
kafkaReadingsTopic: String = readings_prepared
kafkaStatsTopic: String = alert_2_stats
kafkaDedupWatermarkTime: String = 1 minute
joinWatermarkTime: String = 1 minute
bigQueryTargetTable: String = smartplugs.alert_2_anomaly
bigQueryTempBucket: String = pandora-sde-case/alert_2
outputTriggerTime: String = 1 minute


## Define The Required Schema

In [3]:
// This will be used to give the source `readings_prepared` stream data a schema
val readingsSchema = StructType(Seq(
    StructField("message_id", StringType, false),
    StructField("reading_ts", TimestampType, false),
    StructField("reading_value", FloatType, false),
    StructField("reading_type", IntegerType, false),
    StructField("plug_id", IntegerType, false),
    StructField("household_id", IntegerType, false),
    StructField("house_id", IntegerType, false)
))

val statsSchema = StructType(Seq(
    StructField("day", StringType, false),
    StructField("hour", IntegerType, false),
    StructField("mean", FloatType, false),
    StructField("m2", FloatType, false),
    StructField("variance", FloatType, false),
    StructField("std_dev", FloatType, false),
    StructField("count", LongType, false),
    StructField("last_ts", TimestampType, false)
))

readingsSchema: org.apache.spark.sql.types.StructType = StructType(StructField(message_id,StringType,false), StructField(reading_ts,TimestampType,false), StructField(reading_value,FloatType,false), StructField(reading_type,IntegerType,false), StructField(plug_id,IntegerType,false), StructField(household_id,IntegerType,false), StructField(house_id,IntegerType,false))
statsSchema: org.apache.spark.sql.types.StructType = StructType(StructField(day,StringType,false), StructField(hour,IntegerType,false), StructField(mean,FloatType,false), StructField(m2,FloatType,false), StructField(variance,FloatType,false), StructField(std_dev,FloatType,false), StructField(count,LongType,false), StructField(last_ts,TimestampType,false))


### Read and Parse The Input Data Streams

There are 2 input streams this time, `readings_prep` and `alert_1_stats` topic. They will be joined to detect anomalies

In [4]:
// Drop duplicates if seen in an arbitrary watermark. Bounds are necessary so that Spark does not store 
// ALL records in the state memory
val readings = spark
    .readStream 
    .format("kafka")
    .option("kafka.bootstrap.servers", kafkaBootstrapServer)
    .option("subscribe", kafkaReadingsTopic)
    .load()
    .selectExpr("CAST(value AS STRING)")
    .select(from_json($"value", readingsSchema).as("data"))
    .select($"data.*")
    .withWatermark("reading_ts", kafkaDedupWatermarkTime) 
    .dropDuplicates()
    .filter($"reading_type" === 1) // Only take the "current load" measurement

readings: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [message_id: string, reading_ts: timestamp ... 5 more fields]


In [5]:
// We don't need to deduplicate the stats, but we need to drop records with mean=/0
val stats = spark
    .readStream 
    .format("kafka")
    .option("kafka.bootstrap.servers", kafkaBootstrapServer)
    .option("subscribe", kafkaStatsTopic)
    .load()
    .selectExpr("CAST(value AS STRING)")
    .select(from_json($"value", statsSchema).as("data"))
    .select($"data.day", $"data.hour", $"data.mean", $"data.std_dev", $"data.last_ts")
    .filter($"mean" > 0.0)


stats: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [day: string, hour: int ... 3 more fields]


#### Peek at The Input Data Streams

##### Readings

In [6]:
val readingsQuery = readings.writeStream.format("memory").queryName("readings").start()
Thread.sleep(10000)
readingsQuery.status

readingsQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3b36f0b0
res0: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}


In [32]:
spark.sql("select * from readings").show()

+----------+-------------------+-------------+------------+-------+------------+--------+
|message_id|         reading_ts|reading_value|reading_type|plug_id|household_id|house_id|
+----------+-------------------+-------------+------------+-------+------------+--------+
|  47669710|2013-09-01 05:59:20|          0.0|           1|      1|           0|       5|
|  48068742|2013-09-01 06:03:20|          0.0|           1|      1|           0|       5|
|  48268156|2013-09-01 06:05:20|          0.0|           1|      1|           0|       3|
|  48767019|2013-09-01 06:10:20|          0.0|           1|      0|           0|       6|
|  47270293|2013-09-01 05:55:20|          0.0|           1|      1|           0|       7|
|  48401708|2013-09-01 06:06:40|        8.255|           1|      1|           0|       8|
|  47836359|2013-09-01 06:01:00|          0.0|           1|      2|           0|       7|
|  47270400|2013-09-01 05:55:20|          0.0|           1|      2|           0|       9|
|  4730349

In [30]:
// readingsQuery.stop()
readingsQuery.lastProgress

res21: org.apache.spark.sql.streaming.StreamingQueryProgress =
{
  "id" : "e37b32af-0c43-4e6e-9a3f-7794c6959505",
  "runId" : "a93d4ae7-49d2-4c69-b225-8984a8317570",
  "name" : "readings",
  "timestamp" : "2020-03-04T09:20:14.163Z",
  "batchId" : 2,
  "numInputRows" : 5249,
  "inputRowsPerSecond" : 216.4536082474227,
  "processedRowsPerSecond" : 77.15713655740115,
  "durationMs" : {
    "addBatch" : 67546,
    "getBatch" : 0,
    "getEndOffset" : 0,
    "queryPlanning" : 348,
    "setOffsetRange" : 0,
    "triggerExecution" : 68030,
    "walCommit" : 69
  },
  "eventTime" : {
    "avg" : "2013-09-01T06:48:14.932Z",
    "max" : "2013-09-01T07:41:20.000Z",
    "min" : "2013-09-01T05:38:20.000Z",
    "watermark" : "2013-09-01T06:13:20.000Z"
  },
  "stateOperators" : [ {
    "numRowsTotal" ...

In [31]:
readingsQuery.status

res22: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}


##### Stats

In [10]:
val statsQuery = stats.writeStream.format("memory").queryName("stats").start()
statsQuery.status

statsQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6bdb1f4a
res4: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Getting offsets from KafkaV2[Subscribe[alert_2_stats]]",
  "isDataAvailable" : false,
  "isTriggerActive" : true
}


In [33]:
// Thread.sleep(10000)
spark.sql("select * from stats").show()

+--------+----+---------+---------+-------------------+
|     day|hour|     mean|  std_dev|            last_ts|
+--------+----+---------+---------+-------------------+
|20130901|   5|1.8427234|4.0150304|2013-09-01 05:46:40|
|20130901|   6| 9.980184| 40.45797|2013-09-01 06:22:00|
|20130901|   9|29.620712| 86.62796|2013-09-01 09:00:20|
|20130901|   7| 34.88543|  98.4204|2013-09-01 07:46:20|
|20130901|   8|32.391823| 88.62441|2013-09-01 08:35:40|
+--------+----+---------+---------+-------------------+



In [28]:
// statsQuery.stop()
statsQuery.lastProgress

res19: org.apache.spark.sql.streaming.StreamingQueryProgress =
{
  "id" : "6e7bee32-b81b-4f7a-a9d8-e7b7fd1f6ff6",
  "runId" : "22bb80e2-ea08-4f5f-93b7-eace29a7f6b1",
  "name" : "stats",
  "timestamp" : "2020-03-04T09:21:04.502Z",
  "batchId" : 1,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 83.33333333333333,
  "processedRowsPerSecond" : 0.053969453289438175,
  "durationMs" : {
    "addBatch" : 17907,
    "getBatch" : 1,
    "getEndOffset" : 0,
    "queryPlanning" : 121,
    "setOffsetRange" : 1,
    "triggerExecution" : 18529,
    "walCommit" : 36
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[alert_2_stats]]",
    "startOffset" : {
      "alert_2_stats" : {
        "0" : 0
      }
    },
    "endOffset" : {
      "alert_2_stats" : {
        "...

In [29]:
statsQuery.status

res20: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}


## Join Both Data Streams


We can only compare the readings with the past statistics. The `reading_ts` value of `readings_prepared` stream needs to be bigger than the `last_ts` value of the `alert_2_stats` stream. We need to ensure we are comparing the stats from the right `day` and `hour`.

After joining, we need to ensure that we only compare the readings with their latest past statistics.

The `alert_2_stats` stream is assumed to be slower to arrive compared to the readings_prepared stream, since it needs to wait at least 2 records to start calculating, and the default write trigger is every 1 minute. 

In [14]:
// Anomaly detection is done by getting the latest std_dev and mean value
// Then act on it by comparing to the 
val anomaly = readings
    .withWatermark("reading_ts", joinWatermarkTime)
    .join(
        stats.withWatermark("last_ts", joinWatermarkTime),
        // Join conditions
        $"reading_ts" > $"last_ts" &&
        hour($"reading_ts") === $"hour" &&
        date_format($"reading_ts", "yyyyMMdd") === $"day",
        // Join type
        "inner"
    )
    .filter($"reading_value" > $"mean" + $"std_dev")

anomaly: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [message_id: string, reading_ts: timestamp ... 10 more fields]


In [15]:
val peekQuery = anomaly
    .writeStream
    .format("memory")
    .queryName("anomaly")
    .start()

peekQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@48038837


In [34]:
spark.sql("select * from anomaly").show()

+----------+-------------------+-------------+------------+-------+------------+--------+--------+----+---------+--------+-------------------+
|message_id|         reading_ts|reading_value|reading_type|plug_id|household_id|house_id|     day|hour|     mean| std_dev|            last_ts|
+----------+-------------------+-------------+------------+-------+------------+--------+--------+----+---------+--------+-------------------+
|  69012174|2013-09-01 09:33:20|      141.553|           1|      1|           0|       1|20130901|   9|29.620712|86.62796|2013-09-01 09:00:20|
|  69247340|2013-09-01 09:35:40|      396.696|           1|      1|           0|       8|20130901|   9|29.620712|86.62796|2013-09-01 09:00:20|
|  66916542|2013-09-01 09:12:20|      405.686|           1|      1|           0|       8|20130901|   9|29.620712|86.62796|2013-09-01 09:00:20|
|  70376478|2013-09-01 09:47:00|      411.481|           1|      1|           0|       8|20130901|   9|29.620712|86.62796|2013-09-01 09:00:20|

In [26]:
peekQuery.lastProgress

res17: org.apache.spark.sql.streaming.StreamingQueryProgress =
{
  "id" : "0e3c0ee6-6d96-43d4-8963-c147ef0bcedb",
  "runId" : "dece0a41-b90b-46cf-8fb2-aa94949c6b59",
  "name" : "anomaly",
  "timestamp" : "2020-03-04T09:19:59.052Z",
  "batchId" : 0,
  "numInputRows" : 49,
  "processedRowsPerSecond" : 0.6581333190066216,
  "durationMs" : {
    "addBatch" : 72221,
    "getBatch" : 1,
    "getEndOffset" : 0,
    "queryPlanning" : 1565,
    "setOffsetRange" : 495,
    "triggerExecution" : 74453,
    "walCommit" : 82
  },
  "eventTime" : {
    "avg" : "2013-09-01T06:47:10.000Z",
    "max" : "2013-09-01T07:05:00.000Z",
    "min" : "2013-09-01T06:22:00.000Z",
    "watermark" : "1970-01-01T00:00:00.000Z"
  },
  "stateOperators" : [ {
    "numRowsTotal" : 22,
    "numRowsUpdated" : 22,
    "memor...

In [27]:
peekQuery.status

res18: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}


In [19]:
// peekQuery.stop()

## Write to BigQuery

Write to BigQuery once a minute. BigQuery inserts works better with batched data and this ensures that we give enough time for `alert_2_stats` to accumulate. The [BigQuery connector](https://github.com/GoogleCloudDataproc/spark-bigquery-connector) is installed during cluster setup and loaded automatically when spark-shell session is initiated.

In [20]:
val ingestQuery = anomaly
    .writeStream
    .trigger(Trigger.ProcessingTime(outputTriggerTime))
    .foreachBatch{ 
        (batchDF: DataFrame, batchId: Long) =>
            batchDF.write.format("bigquery")
                .option("table", bigQueryTargetTable)
                .option("temporaryGcsBucket", bigQueryTempBucket)
                .mode(SaveMode.Append)
                .save()
    }.start()

ingestQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5716fb32


In [35]:
ingestQuery.lastProgress

res26: org.apache.spark.sql.streaming.StreamingQueryProgress =
{
  "id" : "5e4d4f11-e20b-4cf5-93d1-e40e6b62a316",
  "runId" : "f0e6aa39-58f5-4d09-8db1-22725b4f5db3",
  "name" : null,
  "timestamp" : "2020-03-04T09:24:34.974Z",
  "batchId" : 2,
  "numInputRows" : 23333,
  "inputRowsPerSecond" : 199.5057886006464,
  "processedRowsPerSecond" : 229.6148320179496,
  "durationMs" : {
    "addBatch" : 101243,
    "getBatch" : 1,
    "getEndOffset" : 0,
    "queryPlanning" : 311,
    "setOffsetRange" : 5,
    "triggerExecution" : 101618,
    "walCommit" : 26
  },
  "eventTime" : {
    "avg" : "2013-09-01T19:40:46.472Z",
    "max" : "2013-09-01T23:06:00.000Z",
    "min" : "2013-09-01T15:59:00.000Z",
    "watermark" : "2013-09-01T14:00:40.000Z"
  },
  "stateOperators" : [ {
    "numRowsTotal" : 1...

In [36]:
ingestQuery.status

res27: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}


In [23]:
// ingestQuery.stop()